In [ ]:
import os
from dotenv import load_dotenv
import openai  # 또는 openai 관련 라이브러리

# .env 파일 로드
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 투자요정 프롬프트

In [ ]:
fairy_prompt = """
당신은 어린이 투자 시뮬레이션 게임의 시나리오 작가입니다. 다음 조건에 따라 7턴 분량의 게임 진행 정보를 JSON 배열로 생성하세요.

게임 배경:
- 아이는 마법 마을의 '작은 투자 마법사'입니다.
- 투자 가능한 세 상점은 다음과 같습니다:
  1. 빵집 (저위험):
    - 특징: 마을 사람들이 매일 필요로 하는 빵을 만드는 곳
    - 설명: 매일 필요한 빵이라 망할 확률이 거의 없어요! 하지만 조금씩
  2. 서커스단 (중위험):
    - 특징: 가끔 마을에 와서 공연하는 유명한 서커스단
    - 설명: 인기가 많을 때는 대박이지만, 비올 때는 관객이 적어져요
  3. 마법연구소 (고위험):
    - 특징: 새로운 마법을 개발하는 신비한 연구소
    - 설명: 대발명하면 엄청난 돈을 벌지만, 실패하면 손해가 커요!

### 가격 변동 원칙
모든 종목은 좋은 일이 있으면 상승하고 안 좋은 일이 있으면 하락합니다.
 예) 날씨가 좋아서 빵이 많이 팔렸어요 -> 빵집 상승
    새로운 동물이 연구에 도움이 되지 않았어요 -> 마법 연구소 하락
- **마법 연구소**
  - 실패했을 때는 **급락**도 발생합니다.(최대 -50)
  - 단순히 "연구 계속" 수준의 묘사만으로는 가치를 올릴 수 없습니다.
- **야외 서커스단**
  - 날씨, 외부 환경, 사람들의 관심에 따라 ** 크게 오르거나 내릴 수** 있습니다. (최대 -25)
  - 특별한 공연, 방해 요소에 따라 변동성이 있어야 합니다.
  - 날씨에 따라만 변동되는데 날씨 외의 요소들에 따라서도 변동성이 있도록 조정하세요.
- **빵집**
  - 기본적으로 안정적인 변동폭을 가지되,
  - **강력한 외부 요인** (경쟁, 재료 문제 등)에 의해 **하락도 발생**할 수 있습니다.(최대 -10)
    
### 시나리오 작성 지침
각 턴은 다음 필드를 포함해야 합니다:
- `turn_number`: (int) 현재 턴 번호 (1~7)
- `result`: (str) 반드시 **이전 턴의 `news`에서 예고된 사건이 실제로 일어난 내용만**을 작성하세요. 현실에서 어떻게 벌어졌는지, 각 집에 어떤 영향이 있었는지를 포함해야 합니다.
    - 1턴에서는 게임의 **배경과 목적**을 어린이가 이해할 수 있게 소개합니다.
    - 2턴부터는 반드시 **이전 턴의 `news`에서 예고된 사건이 실제로 어떻게 벌어졌는지**를 설명하세요.
    - 각 집이 어떻게 영향을 받았는지도 함께 서술해주세요. 표현은 아동 친화적으로 간단하고 명확하게 써 주세요.
- `news`: (str)
    - 다음 턴에 발생할 **자연현상, 마을 소문, 동물 출현, 기상 예보, 마법 사건** 등을 창의적으로 작성해 주세요.
    - 명확하고 단순하게, 아동의 상상력을 자극하는 표현을 사용하세요.
    - 예: "내일은 큰 비가 올 예정입니다!"
- `news_hint`: (str)
    - 아이들이 어떤 집이 유리하거나 불리할 수 있을지 추측할 수 있도록 도와주는 **간접적인 힌트**를 제공합니다.
    - 절대 특정 종목의 가격이 오를 것이라는 표현을 하지 마세요.
    - "그럴지도 몰라요", "도움이 될 수 있어요"처럼 **추측을 유도**하는 표현을 사용하세요.
    - 힌트는 오르는 종목보다는 떨어지는 종목을 피할 수 있도록 제공하세요.
    - 예: "비가 많이 오면 야외에서 공연이 어려울 수도 있어요."
- `stocks`: 각 종목의 상태를 다음 요소들과 함께 구성해 주세요:
    - `name`: (str) 종목 이름 (빵집 / 서커스단 / 마법연구소)
    - `initial_value`: (int) 해당 종목의 최초 가치, 초기에 정해지고 절대 변동되지 않아요 (최초 가치는 고위험이 가장 높고 저위험이 가장 낮도록 알아서 설정해주세요.)
    - `risk_level`: (str) 고위험 / 중위험 / 저위험
    - `current_value`: (int) 현재 턴의 result에 따라 변동된 현재 가치
    - `expectation`: (str) 다음 턴 뉴스에 대한 각 집의 예상 반응. 구체적이고 직접적인 힌트로 작성하세요.

### 상승 제한 규칙
- 어떤 종목도 **3턴 이상 연속 상승해서는 안 됩니다.**
    - 단, 특별한 사유가 있다면 3턴 연속 상승은 가능하나, 이후 **반드시 조정(하락 or 정체)**이 발생해야 합니다.

### 시나리오 흥미유발
날씨, 축제 외에도 다양한 유형의 이벤트를 반드시 포함하세요.
단순 반복을 피하고, **예상치 못한 흥미로운 사건**이나 **경쟁**, **기술**, **사회 변화** 등을 활용하여 아이들의 상상력을 자극할 수 있도록 해주세요.    

### 출력 예시 (1턴만, 그대로 쓰지 말고 새로운 내용으로 7턴 전체 생성하세요)
아래의 출력 예시를 참고하되 그대로 사용하지말고 새로운 시나리오를 생성해주세요.
[
  {
    "turn_number": 1,
    "result": "당신은 마을의 어린 투자 마법사가 되어 10일 동안 이 상점들에 마법 코인을 투자하여 마을 사람들을 도와주는 미션을 받았습니다! 코인을 최대한 많이 얻어보세요!",
    "news": "내일은 화창한 날씨가 예상됩니다",
    "news_hint": "어느 상점에 얼마를 투자할까요?",
    "stocks": [
      {
        "name": "빵집",
        "initial_value": 100,
        "risk_level": "저위험",
        "current_value": 100,
        "expectation": "좋은 날씨에는 사람들이 더 많이 나와 빵을 사요"
      },
      {
        "name": "서커스단",
        "initial_value": 80,
        "risk_level": "중위험",
        "current_value": 80,
        "expectation": "맑은 날에는 서커스 관객이 많아져요"
      },
      {
        "name": "마법연구소",
        "initial_value": 50,
        "risk_level": "고위험",
        "current_value": 50,
        "expectation": "날씨는 마법 연구에 큰 영향이 없어요"
      }
    ]
  }
]

※ 이 게임은 5세에서 10세 사이의 아이들을 위한 것이므로, 어려운 경제 용어나 주식 용어는 사용하지 마세요. 

⚠️ 주의사항:
- 매 턴마다 이전 뉴스의 사건이 실제로 벌어졌다는 설명은 반드시 result에 포함되어야 합니다.
- `news`는 매번 새롭고 창의적이어야 하며, 반복되면 안됩니다.
- 전체 출력은 JSON 리스트 형식으로, 설명 없이 순수 데이터만 출력해 주세요.
- 이 게임은 5세에서 10세의 아이들을 위한 것으로 어려운 경제 용어나 주식 용어는 절대 쓰지 마세요. 아이들이 쉽게 이해할 수 있어야 합니다.
- 내용은 동화 속 분위기를 살려서 재밌고 따뜻하게 써주세요.
- 모든 출력은 반드시 한국어로 해주세요.

"""

In [9]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0.6,
    messages=[
        {"role": "system", "content": fairy_prompt},
        {"role": "user", "content": "작은 투자 마법사 기반 7턴짜리 게임 시나리오를 작성해줘. 아이가 공격적인 투자 성향이 강한데 안정적인 자산에 투자해야하는 이유를 배울 수 있도록 시나리오 작성해줘. 그리고 각 뉴스가 어떤 risk_level에 영향을 미치는 뉴스인지 태그 달아줘"}
    ]
)

output_text = response.choices[0].message.content
print(output_text)

with open("fairy.json", "w", encoding="utf-8") as f:
    f.write(output_text)

```json
[
  {
    "turn_number": 1,
    "result": "당신은 마법 마을의 작은 투자 마법사가 되어, 세 가지 상점에 마법 코인을 투자하여 마을 사람들을 도와주는 임무를 맡았습니다! 하지만, 항상 높은 위험을 감수하는 것만이 정답이 아니라는 걸 배워야 해요.",
    "news": "내일은 맑은 날씨가 계속될 예정입니다.",
    "news_hint": "어떤 상점에 투자할까요? 안정적인 선택이 좋을지도 몰라요.",
    "stocks": [
      {
        "name": "빵집",
        "initial_value": 100,
        "risk_level": "저위험",
        "current_value": 100,
        "expectation": "맑은 날에는 빵을 사러 오는 사람들이 많아요."
      },
      {
        "name": "서커스단",
        "initial_value": 80,
        "risk_level": "중위험",
        "current_value": 80,
        "expectation": "날씨가 좋으면 서커스 관객이 많아질 거예요."
      },
      {
        "name": "마법연구소",
        "initial_value": 50,
        "risk_level": "고위험",
        "current_value": 50,
        "expectation": "날씨는 연구에 큰 영향을 주지 않아요."
      }
    ]
  },
  {
    "turn_number": 2,
    "result": "내일의 맑은 날씨 덕분에 빵집의 빵이 많이 팔렸고, 서커스단도 많은 관객을 맞이했어요! 하지만 마법연구소는 새로운 마법을 개발하는 데 어려움을 겪었어요.",
    "news": "서커스단이 특별한 공연을 준비하고 있다고 해요!",
    "news_hin

# 아기돼지 프롬프트

In [ ]:
# pig_prompt = """
# 당신은 미취학 아동을 대상으로 하는 모바일 모의투자 게임 튜토리얼 시나리오 디자이너입니다.
# 다음 안내사항에 따라 시나리오를 구상하고 JSON 리스트 형식의 데이터를 출력해주세요.

# ### 게임 기본 구성
# 1. 이 게임은 동화 '아기돼지 삼형제'속 요소를 사용하여 아이들이 이해할 수 있는 모의투자 튜토리얼 시나리오를 구성합니다.
# 2. 투자 종목은 지푸라기 집, 나무 집, 벽돌 집 입니다. 세가지 종목을 고위험 고수익, 균형형, 장기 안정형 세가지 유형(`risk_level`)으로 지정해주세요.
#   - 고위험 고수익 유형의 종목이 "고수익"을 낼 수 있는 턴이 무조건 하나 포함되어야 합니다.
# 3. 이 게임은 턴제로 운영되고, 총 5턴으로 시나리오를 구성해주세요.
# 4. 매 턴 마다 다음 턴에 발생할 랜덤 이벤트를 미리 예고해주는 뉴스를 보여줍니다. 이때 이벤트는 각 투자 종목에 호재 또는 악재이어야 합니다. **각 종목별로 호재인 이벤트를 골고루 포함**해주세요
# 5. 각 투자 종목은 해당 턴에 발생한 이벤트에 의해 값이 변동합니다. 종목의 유형별로 변동하는 값은 명확히 차이가 나야하고, 아이들이 납득할 수 있도록 왜 값이 변동했는지 해당 턴에 발생한 이벤트와 연관지어 그 이유를 설명해주세요.

# ### 필수사항
# 1. 아이들이 이해할 수 있는 쉬운 단어만 사용해주세요. 한자어와 전문적인 단어는 제외하세요.
# 2. 친절한 말투를 사용해주세요. (해요체나 습니다체를 사용하세요.)
# 3. 시나리오는 논리적이어야 합니다. 아이들이 납득할 수 있도록 시나리오를 구성하세요. 어떤 일에는 항상 그 이유가 같이 설명되어야 합니다.
# 4. 다른 설명없이 오직 JSON 형식의 데이터만 출력하세요
# 5. 모든 투자 종목의 가격이 0이 되지 않도록 시나리오를 구성하세요.
#   - ex. 지푸라기 집이 바람에 날라가서 없어졌어요! -> 지푸라기 집이 바람에 날라가서 없어졌지만, 빠르게 고쳐서 다시 집을 지었어요! 하지만 사람들은 안전하지 않은 지푸라기 집을 좋지 않다고 생각해요.

# ### 가격 변동 원칙
# 모든 종목은 좋은 일이 있으면 상승하고 안 좋은 일이 있으면 하락합니다.
# - **지푸라기집**
#   - 실패했을 때는 **급락**도 발생합니다.(최대 -50)
#   - 다른 집들에 비해 상승, 하락을 자주 하고 큰 폭으로 변동됩니다.
#   - 너무 하락만 하지 않고 상승도 하는 요소들을 넣어주세요.
# - **나무집**
#   - 날씨, 외부 환경, 사람들의 관심에 따라 ** 크게 오르거나 내릴 수** 있습니다. (최대 -25)
#   - 특별한 행사, 방해 요소에 따라 변동성이 있어야 합니다.
# - **벽돌집**
#   - 기본적으로 안정적인 변동폭을 가지되,
#   - **강력한 외부 요인** (경쟁, 재료 문제 등)에 의해 **하락도 발생**할 수 있습니다.(최대 -10)
#   - 계속 변동이 없이 유지되는데 지속적으로 작지만 상승을 해야 합니다.

# ### 출력JSON 형식
# JSON은 총 5개 턴의 정보를 담고 있는 리스트여야 합니다.
# 각 턴 객체는 다음 키를 가져야 합니다:
# - `turn_number`: (정수) 현재 턴 번호 (1부터 5까지).
# - `result`: (문자열)
#   - 1턴에서는 아이들이 무엇을 위해 각 종목을 사고 팔아야하는지 이해할 수 있도록 배경을 간단히 설명해주세요.
#      - ex. "오늘은 아기돼지 삼형제가 각자 다른 집을 지었어요! 첫째 돼지는 지푸라기 집을 지었고, 둘째 돼지는 나무집을, 셋째 돼지는 벽돌집을 지었죠. 지푸라기 집은 싸지만 쉽게 무너지고, 나무집은 조금 튼튼하지만 여전히 위험해요. 벽돌집은 비싸지만 아주 튼튼하답니다. 늑대가 나타나도 무사히 살 수 있고, 시간이 지나면 더 비싸지는 집은 무엇일까요? 어떤 집에 투자해야 돈도 벌고 안전하게 지낼 수 있을까요? 아기돼지들의 집을 사고팔며 똑똑한 선택을 해보아요!"
#   - 2턴부터는 반드시 이전 턴의 `news`에서 예고된 사건이 **현재 시점에서 실제로 어떻게 벌어졌는지**을 현재시제로 작성하세요. ex. 둘째 돼지 집에 불이 났어요!
#      - 이전 턴의 `news`에서 예보하지 않은 일은 절대 일어나지 않습니다. 이전 턴의 `news`에서 언급하지 않은 내용은 말하지 마세요.
#      - 미래를 의미하는 단어(내일, 다음 턴)는 절대 사용하지 마세요.
#      - 각 종목이 어떻게 영향을 받았는지도 함께 서술해주세요
# - `news`: (문자열) 다음 턴에 발생할 사건을 예고하세요. (자연현상, 마을 행사, 동물 출현, 소문 등 객관적 사건) ex. 내일은 늑대가 나타날 수도 있어요
# - `news_hint`: (문자열)
#   - 아이들이 어떤 집이 유리하거나 불리할 수 있을지 추측할 수 있도록 도와주는 **간접적인 힌트**를 제공합니다.
#   - 절대 특정 종목의 가격이 오를 것이라는 표현을 하지 마세요.
#   - "그럴지도 몰라요", "도움이 될 수 있어요"처럼 **추측을 유도**하는 표현을 사용하세요.
#   - 힌트는 오르는 종목보다는 떨어지는 종목을 피할 수 있도록 제공하세요.
#     - 예: "비가 많이 오면 지푸라기 집이 젖어서 무너질 수도 있어요."
#   - 절대 종목 이름을 사용하지 마세요. 종목을 말해야 할 때는 최대한 그 종목의 특징을 서술하여 빗대어 설명해주세요
#     - 예: "벽돌집은 늑대의 공격에 버틸지도 몰라요!"(X) -> "튼튼한 집은 늑대의 공격에도 버틸 수 있을거에요!"(O)
# - `stocks`: (리스트) 각 투자 종목의 상세 정보를 담은 객체들의 리스트. 각 종목 객체는 다음 키를 가져야 합니다:
#   - `name`: (문자열) 종목 이름 .
#   - `expectation`: (문자열)
#     - 각 턴에서 예보된 뉴스 `news`에 대해 해당 종목이 어떤 장점과 단점을 갖는지 구분하여 한 문장으로 설명하세요.
#     - 가치 변동에 대한 어떠한 언급도 하지 않습니다.
#     - 예를 들어 집이 인기가 있을 것 같다고 설명을 할 때는 왜 인기가 있을 것 같은지도 같이 설명하세요.
#   - `initial_value`: (정수) 해당 종목의 최초 가치, 초기에 정해지고 절대 변동되지 않아요 (최초 가치는 고위험이 가장 높고 저위험이 가장 낮도록 알아서 설정해주세요.)
#   - `current_value`: (정수) 해당 턴 종료 시점의 현재 가치.
#     - 이 값은 `result`를 반영하여 논리적으로 변동되어야 합니다. 다음 턴에 일어날 일을 의미하는 같은 턴의 `news`에는 영향을 받지 않습니다.
#     - `risk_level`별로 변동폭이 달라야합니다. 세 유형의 가격 변동 폭은 눈에 띄게 차이가 나야합니다. 고위험고수익이 가장 높은 변동성을가지고, 균형형, 장기 안정형 순으로 점차 변동성이 낮아집니다.
#   - `risk_level`: (문자열) 해당 종목의 위험도
# """

In [45]:
pig_prompt = """
당신은 미취학 아동을 위한 투자 게임 시나리오 작가입니다. '아기돼지 삼형제' 동화를 바탕으로 아이들이 쉽게 이해할 수 있는 5턴 분량의 모의 투자 게임 시나리오를 아래 규칙에 따라 JSON 형식으로 출력해주세요.

### 기본 설정
- 투자 대상: 지푸라기 집 (고위험 고수익), 나무 집 (균형형), 벽돌 집 (장기 안정형)
- 각 집은 매 턴마다 사건의 영향으로 가치가 변동됩니다. current_value는 해당 턴의 result에 따라 변화된 값으로 지정하세요.
- 모든 집의 initial_value는 턴 1에서 정해지며 턴마다 변하지 않습니다.

### 시나리오 규칙
- 총 5턴으로 구성된 JSON 리스트를 생성하세요.
- 각 턴에는 다음 키를 포함한 JSON 객체를 생성해야 합니다:

### 가격 변동 원칙
첫번째 턴에서는 initial_value와 current_value가 같아야 합니다.
모든 종목은 좋은 일이 있으면 상승하고 안 좋은 일이 있으면 하락합니다.
- **지푸라기집**
  - 실패했을 때는 **급락**도 발생합니다.(최대 -50)
  - 다른 집들에 비해 상승, 하락을 자주 하고 큰 폭으로 변동됩니다.
  - 너무 하락만 하지 않고 상승도 하는 요소들을 넣어주세요.
- **나무집**
  - 날씨, 외부 환경, 사람들의 관심에 따라 ** 크게 오르거나 내릴 수** 있습니다. (최대 -25)
  - 특별한 행사, 방해 요소에 따라 변동성이 있어야 합니다.
- **벽돌집**
  - 기본적으로 안정적인 변동폭을 가지되,
  - **강력한 외부 요인** (경쟁, 재료 문제 등)에 의해 **하락도 발생**할 수 있습니다.(최대 -10)
  - 계속 변동이 없이 유지되는데 지속적으로 작지만 상승을 해야 합니다.

### 출력 형식 (각 턴마다):

- `turn_number`: 현재 턴 번호 (1~5)
- `result`: (턴 1) 아기돼지 삼형제와 각 집의 차이, 아이가 왜 투자해야 하는지를 설명 / (턴 2~5) 반드시 직전 턴의 `news`에서 예고된 사건이 실제로 일어난 결과를 설명
- `news`: 다음 턴에 발생할 사건 예보 (예: "내일은 비가 온대요!")
- `news_hint`: 어떤 집이 유리하거나 불리할 수 있을지 간접적인 힌트 (종목 이름 사용 금지, 예: "젖기 쉬운 집은 위험할 수도 있어요")
- `stocks`: 3개 집의 상태 리스트. 각 항목에 다음 키를 포함하세요:
  - `name`: 집 이름 ("지푸라기 집" / "나무 집" / "벽돌 집")
  - `risk_level`: 고위험 고수익 / 균형형 / 장기 안정형
  - `initial_value`: 첫 턴에 지정한 고정 가치 (지푸라기 집 가장 높고, 벽돌 집 가장 낮음)
  - `current_value`: 이번 턴 결과 반영된 현재 가치 (각 위험도에 따라 변동폭 달라야 함)
  - `expectation`: 현재 `news`에 따라 이 집이 어떤 영향을 받을 수 있는지 간접적으로 설명 (가격 언급 금지)

### 추가 조건
- 쉬운 말만 사용해주세요. 해요체 사용.
- 가격은 절대 0이 되지 않도록 하세요.
- 각 집에 호재와 악재가 골고루 분포되도록 하세요.
- 고위험 종목(지푸라기 집)은 최소 1턴 이상에서 크게 상승해야 합니다.

### 출력 예시 금지
- 설명 없이 반드시 JSON 리스트만 출력하세요.

"""

In [46]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0.6,
    messages=[
        {"role": "system", "content": pig_prompt},
        {"role": "user", "content": "아기돼지 삼형제를 기반으로 5턴짜리 게임 시나리오를 작성해줘"}
    ]
)

output_text = response.choices[0].message.content
print(output_text)

with open("pig.json", "w", encoding="utf-8") as f:
    f.write(output_text)

APIConnectionError: Connection error.